In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 1. Some lil stats

In [2]:
train = pd.read_csv("data/train.csv", sep=",")

In [3]:
# train.head(3)

In [4]:
#train.dtypes

#Mtrain.describe().T

#{"Survived":np.count_nonzero(train.Survived[train.Survived==1]), "Dead":len(train.Survived[train.Survived==0])}

In [5]:
# NaN Sources
train.columns[train.isna().any()]

Index(['Age', 'Cabin', 'Embarked'], dtype='object')

In [6]:
# NaN ratios
np.array([train.Age.isna().sum(),train.Cabin.isna().sum(),train.Embarked.isna().sum()])/train.shape[0]

array([0.1986532 , 0.77104377, 0.00224467])

# 2. Imputing missing values

## Function lab

In [7]:
def impute_missing_values(df):
    for col in df.columns:
        if df[col].dtype == np.float:
            df[col] = df[col].fillna(df[col].median())
        if df[col].dtype == np.object:
            df[col] = df[col].fillna(train[col].mode()[0])
    return df

## Function in action

In [8]:
X = impute_missing_values(train)

# 3. Modeling

## To start

In [9]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

In [10]:
X, y = pd.DataFrame([train.SibSp, train.Fare, train.Parch]).T, train.Survived
X.shape, train.shape, y.shape

((891, 3), (891, 12), (891,))

## Adding variables, dummt encoding

### Functions lab

In [11]:
# Age categorical split
def is_child(v, age=10):
    v = np.where(v<age,"yes","no")
    return pd.Series(v, name="Age")

# Create matrix by adding columns with same height
def create_matrix(features, make_dummies=[]):
    variables = []
    for idx, f in enumerate(features):
        if make_dummies[idx]==True:
            variables.append(pd.get_dummies(f, prefix=f.name))
        else: variables.append(f)
    return pd.concat(variables, axis=1)

# Get titles
def get_passager_title(p_names):
    titles = []
    for idx, n in p_names.iteritems():
        titles.append(n.split(",")[1].split(".")[0].strip())
    return pd.Series(titles, name="Name")

# Check surnames
def has_surname(p_names):
    return p_names.apply(lambda x: "(" in x)

### Functions in action

In [12]:
#create_matrix(
    #[train.SibSp,train.Fare,train.Parch,train.Pclass,is_child(train.Age),train.Sex],
    #[False,False,False,True,True,True]).tail(5)

In [31]:
pclass_to_dummies = pd.get_dummies(train.Pclass, prefix="Pclass")
sex_to_dommies = pd.get_dummies(train.Sex, prefix="Sex")
age_to_dummies = pd.get_dummies(is_child(train.Age.values), prefix="Age")
X = pd.concat([X, pclass_to_dummies, sex_to_dommies, age_to_dummies], axis=1)
#X.head()

## Let's train

In [13]:
def train_and_accuracy(x_train, y_train, x_test, y_test):
    model_lr = LogisticRegressionCV()
    model_lr.fit(x_train,y_train)
    return accuracy_score(model_lr.predict(x_test), y_test)

In [14]:
X = create_matrix(
    [train.SibSp,train.Fare,train.Parch,train.Pclass,
     is_child(train.Age),train.Sex,get_passager_title(train.Name), has_surname(train.Name)],
    [False,False,False,True,True,True, True, True])

In [15]:
# Spliting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# Feeding the beast
train_and_accuracy(X_train, y_train, X_test, y_test)

0.8171641791044776

# Best train partition search

## Function lab

In [16]:
def best_train_partition_search(x, y):
    test_size_grid = 6*np.logspace(-2,-1,10)
    test_score_grid = []
    for i in test_size_grid:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=102)
        model_lr = LogisticRegressionCV()
        model_lr.fit(X_train,y_train)
        test_score_grid.append(accuracy_score(model_lr.predict(X_test), y_test))
    return test_size_grid, test_score_grid

## Function in action

In [17]:
import matplotlib.pyplot as plt

test_size_grid, test_score_grid = best_train_partition_search(X, y)
plt.plot(test_size_grid, test_score_grid)

In [18]:
np.mean(test_score_grid)

0.8402847652462683

# Let's do it by cross_validation

In [19]:
model_lr = LogisticRegressionCV()
model_lr.fit(X_train,y_train)
cx_score = cross_val_score(model_lr, X, y, cv=10)
np.mean(cx_score)

0.8328180683236862

# Random Forest

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

n_estimator,
max_depth,
min_sample_split,
min_samplr_leaf

In [71]:
pgrid = {
    "n_estimators": np.arange(50,500,4),
    "max_depth": [4,6,8,10],
    "min_samples_split": [2,4,6],
    "min_samples_leaf": [2,3,4,5,6,7,8,9,10]
}

gforest = GridSearchCV(model_rf, param_grid=pgrid, n_jobs=3, cv=3, verbose=1)
gforest.fit(X, y)

Fitting 3 folds for each of 2484 candidates, totalling 7452 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    7.6s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   38.5s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  1.5min
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed:  2.7min
[Parallel(n_jobs=3)]: Done 1244 tasks      | elapsed:  4.3min
[Parallel(n_jobs=3)]: Done 1794 tasks      | elapsed:  6.1min
[Parallel(n_jobs=3)]: Done 2444 tasks      | elapsed:  8.4min
[Parallel(n_jobs=3)]: Done 3194 tasks      | elapsed: 11.4min
[Parallel(n_jobs=3)]: Done 4044 tasks      | elapsed: 14.2min
[Parallel(n_jobs=3)]: Done 4994 tasks      | elapsed: 17.5min
[Parallel(n_jobs=3)]: Done 6044 tasks      | elapsed: 21.0min
[Parallel(n_jobs=3)]: Done 7194 tasks      | elapsed: 24.9min
[Parallel(n_jobs=3)]: Done 7452 out of 7452 | elapsed: 25.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='entropy',
                                              max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=Tr...e,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=3,

In [75]:
gforest.best_params_

{'max_depth': 4,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 170}

# With best parameters selected

In [77]:
model_rf = RandomForestClassifier(criterion="gini", n_estimators=170, 
                                  min_samples_leaf=2, max_depth=4, 
                                  min_samples_split=2,oob_score=True, )
model_rf.fit(X_train,y_train)
accuracy_score(model_rf.predict(X_test), y_test)

0.7947761194029851